In [2]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import time
import re
import os
from random import choice

# disable logs for webdriver_manager
os.environ['WDM_LOG_LEVEL'] = '0'

import scrapy

from concurrent import futures

In [3]:
class BlogSpider(scrapy.Spider):
    name = 'blogspider'
    start_urls = ['https://www.zyte.com/blog/']

    def parse(self, response):
        for title in response.css('.oxy-post-title'):
            yield {'title': title.css('::text').get()}

        for next_page in response.css('a.next'):
            yield response.follow(next_page, self.parse)

In [10]:
x = BlogSpider()

In [11]:
x.parse()

<BlogSpider 'blogspider' at 0x13316f29420>

In [5]:
def search_city(add):
    try:
        split_list = add.split(', ')
        state = split_list[-1].split(' ')
        state = " ".join(state[:-1])
        city = split_list[-2]
        return city, state
    except Exception as e:
        print(e)
        return '', ''

In [6]:
def extract_stores(new_file_name):
    start_index, final_index = [int(i) for i in new_file_name.split('-')]
    print(datetime.now().strftime("%H:%M:%S"), end=" : ")
    print("STARTING", new_file_name)

    add_null_count = 0

    data = {
        'Search Keyword': [],
        'Storefront Link': [],
        'Company Name': [],
        'Address': [],
        'State': [],
        'City': [],
        'Pincode': [],
        'Phone': [],
        'Email Id': [],
        'Website': [],
        'Rating': [],
        'Total Reviews': [],
        'Products': [],
    }

    s = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s, options=op)

    for link in all_links[start_index:final_index+1]:
        full_key = link.split('q=')[1].replace('+', ' ')

        try:
            driver.get(link)
            time.sleep(8)

        # link error
        except:
            print(datetime.now().strftime("%H:%M:%S"), end=" : ")

            stop_index = all_links.index(link)

            print("driver.get(): %d-%s" %(stop_index, full_key))
            
            # save if error occurs
            if stop_index == start_index:
                save_excel_file(data, folder='raw data', filename=f'{start_index}-{stop_index}')
            else:
                save_excel_file(data, folder='raw data', filename=f'{start_index}-{stop_index-1}')

            delete_excel_file(folder='raw data', filename=new_file_name)

            break

        # address error
        if add_null_count == 15:
            c = 0
            for x in data['Address'][0:15]:
                if isinstance(x, float) or x == '':
                    c += 1
            if c == 15:
                print(datetime.now().strftime("%H:%M:%S"), end=" : ")

                stop_index = all_links.index(link)

                print("no addresses: %d-%s" %(stop_index, full_key))
            
                # save if error occurs
                if stop_index == start_index:
                    save_excel_file(data, folder='raw data', filename=f'{start_index}-{stop_index}')
                else:
                    save_excel_file(data, folder='raw data', filename=f'{start_index}-{stop_index-1}')

                delete_excel_file(folder='raw data', filename=new_file_name)

                break

        # google captcha error
        if 'sorry' in driver.current_url:
            print(datetime.now().strftime("%H:%M:%S"), end=" : ")

            stop_index = all_links.index(link)

            print("google captcha: %d-%s" %(stop_index, full_key))
            
            # save if error occurs
            if stop_index == start_index:
                save_excel_file(data, folder='raw data', filename=f'{start_index}-{stop_index}')
            else:
                save_excel_file(data, folder='raw data', filename=f'{start_index}-{stop_index-1}')

            delete_excel_file(folder='raw data', filename=new_file_name)

            break

        try:
            # all companies
            all_eles = driver.find_elements(By.TAG_NAME, 'div')
            all_eles = [i for i in all_eles if i.get_attribute('jsname') == 'GZq3Ke']

        # elements not found
        except:
            print(datetime.now().strftime("%H:%M:%S"), end=" : ")

            stop_index = all_links.index(link)

            print("find elements: %d-%s" %(stop_index, full_key))
            
            # save if error occurs
            if stop_index == start_index:
                save_excel_file(data, folder='raw data', filename=f'{start_index}-{stop_index}')
            else:
                save_excel_file(data, folder='raw data', filename=f'{start_index}-{stop_index-1}')

            delete_excel_file(folder='raw data', filename=new_file_name)

            break

        for ele in all_eles:
            try:
                ele.click()
                time.sleep(3)

                # company name
                try:
                    name = driver.find_element(By.CLASS_NAME, 'qrShPb').text.title()
                except:
                    name = ''
                    continue
                data['Company Name'].append(name)
                data['Search Keyword'].append(full_key)
                data['Storefront Link'].append(driver.current_url)

                # website
                try:
                    web = driver.find_element(By.CLASS_NAME, 'CL9Uqc')
                    if web.text != 'Website':
                        web = ''
                    else:
                        web = web.get_attribute('href')
                except:
                    web = ''
                data['Website'].append(web)

                # address + phone
                try:
                    ad_ph = driver.find_elements(By.CLASS_NAME, 'LrzXr')

                    if len(ad_ph) > 1:
                        ad = ad_ph[0].text
                        ph = ad_ph[1].text
                        pin = re.findall('\d\d\d\d\d\d', ad)[0]
                        city, state = search_city(ad)

                    elif ad_ph[0].text.replace(' ', '').isdigit():
                        ad = ''
                        ph = ad_ph[0].text
                        pin = ''
                        city = ''
                        state = ''
                        add_null_count += 1

                    else:
                        ad = ad_ph[0].text
                        ph = ''
                        pin = re.findall('\d\d\d\d\d\d', ad)[0]
                        city, state = search_city(ad)
                except:
                    ad = ''
                    ph = ''
                    pin = ''
                    city = ''
                    state = ''
                    add_null_count += 1

                data['Address'].append(ad)
                data['Phone'].append(str(ph))
                data['Pincode'].append(str(pin))
                data['City'].append(city)
                data['State'].append(state)

                # rating
                try:
                    rate = driver.find_element(By.CLASS_NAME, 'Aq14fc').text
                except:
                    rate = ''
                data['Rating'].append(str(rate))

                # reviews
                try:
                    num_r = driver.find_element(By.CLASS_NAME, 'hqzQac').text
                    num_r = int(num_r.split(' ')[0])
                except:
                    num_r = ''
                data['Total Reviews'].append(str(num_r))

                # products
                try:
                    prods = driver.find_elements(By.CLASS_NAME, 'zPcHee')
                    if len(prods) > 0:
                        prods = [i.text for i in prods]
                        prods = ', '.join(prods)
                    else:
                        prods = ''
                except:
                    prods = ''
                data['Products'].append(prods)

            except:
                pass

            save_excel_file(data, folder='raw data', filename=new_file_name)
        save_excel_file(data, folder='raw data', filename=new_file_name)
    save_excel_file(data, folder='raw data', filename=new_file_name)

    driver.quit()

In [7]:
df = pd.read_excel('info.xlsx')
keywords = df['mcat_keywords'].dropna().tolist()
# cities = df['Gl_City_Name'].dropna().tolist()
# states = df['Gl_State_Name'].dropna().tolist()
# types = df['Business Type'].dropna().tolist()
all_links = pd.read_csv('all keyword links.csv').iloc[:, 0].tolist()

# print('keywords       :', len(keywords))
# print('cities         :', len(cities))
# print('types          :', len(types))
print('total searches :', len(all_links))

total searches : 2700000


In [8]:
# print(all_links[25920].split('q=')[1].replace('+', ' '))
# print(all_links[25980].split('q=')[1].replace('+', ' '))

In [9]:
for i in range(46, 55):
    print('%2d %33s %5d-%d' % (i, keywords[i], 540*i, 540*(i+1)-1))

46                 Plastic Die Mould 24840-25379
47               Cup Filling Machine 25380-25919
48    Transformer Oil Filter Machine 25920-26459
49            Fire Ball Extinguisher 26460-26999
50            Garlic Peeling Machine 27000-27539
51        Digital Humidity Indicator 27540-28079
52                  Flameproof Paint 28080-28619
53                        VRF System 28620-29159
54     Concrete Brick Making Machine 29160-29699


In [10]:
def create_file_names(start, finish):
    div = (finish-start+1) // 9

    names = [
        f'{start}-{start+div}',
        f'{start+div+1}-{start+div*2+1}',
        f'{start+div*2+2}-{start+div*3+2}',
        f'{start+div*3+3}-{start+div*4+3}',
        f'{start+div*4+4}-{start+div*5+4}',
        f'{start+div*5+5}-{start+div*6+5}',
        f'{start+div*6+6}-{start+div*7+6}',
        f'{start+div*7+7}-{start+div*8+7}',
        f'{start+div*8+8}-{finish}'
    ]

    return names

In [11]:
# file_names = ['27077-27121']
# file_names.extend(create_file_names(27540, 28079))
# file_names.extend(create_file_names(28080, 28619))
# file_names.extend(create_file_names(28620, 29159))
# file_names.extend(create_file_names(29160, 29699))


# for a in file_names:
#     x, y = [int(i) for i in a.split('-')]
#     print('%11s = %3d'  %(a, y-x+1))

In [13]:
# setting options
op = ChromeOptions()
op.add_argument("log-level=3")
op.add_argument("--disable-gpu")
op.add_argument("--no-sandbox")
op.add_argument("enable-automation")
op.add_argument("--disable-infobars")

op.add_argument("--disable-dev-shm-usage")
op.add_argument("--disable-dev-shm-usage")
op.add_argument("--headless")

In [ ]:
file_names = []
file_names.extend(create_file_names(29390, 29699))

while True:
    # checking null addresses and missing indexes
    dirpath = "raw data/"
    raw_data = [s for s in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, s))]
    raw_data.sort(key=lambda x: int(x.split('-')[0]))

    for i in range(len(raw_data)):
        df = pd.read_excel(dirpath+raw_data[i], index_col=0, dtype=str)
        if df.Address.isnull().values.all(axis=0):
            file_names.append(raw_data[i])
    
    for i in range(len(raw_data)-1):
        file = raw_data[i]
        next_file = raw_data[i+1]

        s1, e1 = [int(i) for i in file.replace('.xlsx', '').split('-')]
        s2, e2 = [int(i) for i in next_file.replace('.xlsx', '').split('-')]

        if e1 == (s2-1):
            pass
        else:
            file_names.append(f"{e1+1}-{s2-1}")

    if len(file_names) < 1:
        break

    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(extract_stores, file_names)

print(datetime.now().strftime("%H:%M:%S"), end=" : ")
print("FINISHED ALL")

07:18:27 : STARTING 29390-29424
07:18:27 : 07:18:27 : STARTING 29460-29494
STARTING 29425-29459
07:30:55 : google captcha: 29436-Concrete Brick Making Machine Wholesalers in Chandigarh
07:30:57 : STARTING 29495-29529
07:30:59 : google captcha: 29471-Concrete Brick Making Machine Wholesalers in New Delhi
07:31:02 : STARTING 29530-29564
07:31:35 : google captcha: 29531-Concrete Brick Making Machine Retailers in Surat
07:31:37 : STARTING 29565-29599
07:31:37 : google captcha: 29401-Concrete Brick Making Machine Exporters in Jalandhar
07:31:40 : STARTING 29600-29634
07:32:24 : google captcha: 29566-Concrete Brick Making Machine Service Providers in Kanpur
07:32:26 : STARTING 29635-29669
07:32:31 : google captcha: 29496-Concrete Brick Making Machine Retailers in Delhi
07:32:33 : STARTING 29670-29699
07:33:14 : google captcha: 29601-Concrete Brick Making Machine Traders in Coimbatore
07:33:16 : STARTING 27077-27121
07:33:52 : google captcha: 29636-Concrete Brick Making Machine Traders in Pon

07:54:27 : google captcha: 29565-Concrete Brick Making Machine Service Providers in Kanchipuram
07:54:29 : STARTING 29635-29669
07:54:30 : STARTING 29671-29669
07:54:32 : google captcha: 29600-Concrete Brick Making Machine Traders in Chinchwad
07:54:33 : STARTING 29700-29670
07:54:41 : google captcha: 29635-Concrete Brick Making Machine Traders in Panipat
07:54:52 : STARTING 29390-29424
07:54:52 : STARTING 29425-29459
07:54:52 : STARTING 29460-29494
07:55:04 : google captcha: 29390-Concrete Brick Making Machine Exporters in Faridabad
07:55:04 : google captcha: 29425-Concrete Brick Making Machine Exporters in Thiruvananthapuram
07:55:04 : google captcha: 29460-Concrete Brick Making Machine Wholesalers in Kolkata
07:55:06 : STARTING 29495-29529
07:55:06 : STARTING 29530-29564
07:55:07 : STARTING 29565-29599
07:55:18 : google captcha: 29565-Concrete Brick Making Machine Service Providers in Kanchipuram
07:55:18 : google captcha: 29530-Concrete Brick Making Machine Retailers in Secunderaba

07:59:46 : STARTING 29565-29530
07:59:49 : STARTING 29530-29564
07:59:50 : google captcha: 29495-Concrete Brick Making Machine Retailers in Dehradun
07:59:51 : google captcha: 29496-Concrete Brick Making Machine Retailers in Delhi
07:59:52 : STARTING 29531-29564
07:59:54 : STARTING 29566-29564
07:59:57 : STARTING 29600-29565
08:00:00 : google captcha: 29530-Concrete Brick Making Machine Retailers in Secunderabad
08:00:00 : STARTING 29565-29599
08:00:02 : STARTING 29566-29599
08:00:04 : google captcha: 29531-Concrete Brick Making Machine Retailers in Surat
08:00:06 : STARTING 29601-29599
08:00:09 : STARTING 29635-29600
08:00:12 : google captcha: 29565-Concrete Brick Making Machine Service Providers in Kanchipuram
08:00:12 : STARTING 29600-29634
08:00:13 : google captcha: 29566-Concrete Brick Making Machine Service Providers in Kanpur
08:00:14 : STARTING 29601-29634
08:00:15 : STARTING 29636-29634
08:00:19 : STARTING 29670-29635
08:00:22 : STARTING 29635-29669
08:00:24 : google captcha: 

In [14]:
with futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(extract_stores, file_names)

print(datetime.now().strftime("%H:%M:%S"), end=" : ")
print("FINISHED ALL")

21:02:41 : STARTING 27000-27060
21:02:41 : STARTING 27061-27121
21:02:41 : STARTING 27122-27182
21:13:42 : google captcha: 27077-27061-27121 Garlic Peeling Machine Suppliers in Indore
21:13:45 : STARTING 27183-27243
21:31:05 : STARTING 27244-27304
21:48:21 : STARTING 27305-27365
21:54:18 : STARTING 27366-27426
22:16:56 : STARTING 27427-27487
22:40:43 : STARTING 27488-27539
23:06:04 : FINISHED ALL


In [16]:
# checking if all files were created
dirpath = "raw data/"
raw_data = [s for s in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, s))]
raw_data.sort(key=lambda x: int(x.split('-')[0]))

count = 1
for i in range(len(raw_data)-1):
    file = raw_data[i]
    next_file = raw_data[i+1]

    s1, e1 = [int(i) for i in file.replace('.xlsx', '').split('-')]
    s2, e2 = [int(i) for i in next_file.replace('.xlsx', '').split('-')]
#     print(e1, s2)
    if e1 == (s2-1):
        count += 1
    else:
        print(file, next_file)

print(count, len(raw_data))
count == len(raw_data)

110 110


True

## Checking totals

In [60]:
dirpath = "raw data/"
raw_data = [s for s in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, s))]

temp = [pd.read_excel(dirpath+i, index_col=0, dtype=str) for i in raw_data]
df = pd.concat(temp)

print(df.shape)

(18738, 13)


In [ ]:
dirpath = "clean data/"
clean_data = [s for s in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, s))]

temp = [pd.read_excel(dirpath+i, index_col=0, dtype=str) for i in clean_data]
df = pd.concat(temp)

print(df.shape)

In [25]:
dirpath = "final data\\merged\\"
final_data = [s for s in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, s))]
final_data = sorted(final_data, key=lambda x: os.path.getmtime(os.path.join(dirpath, x)))

print(final_data)
temp = [pd.read_excel(dirpath+i, index_col=0, dtype=str).shape[0] for i in final_data]

print(sum(temp))

['Paper Testing Machine.xlsx', 'Plastic Testing Machine.xlsx', 'Tile Abrasion Testing Machine.xlsx', 'Wire Testing Machine.xlsx', 'Leader Check Valve.xlsx', 'Microdermabrasion System.xlsx']


16792